In [ ]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader('attention.pdf')
docs = loader.load()
docs

In [ ]:
from langchain.text_splitter import  RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
text_splitter.split_documents(docs)[:5]

In [ ]:
documents =text_splitter.split_documents(docs)
documents

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

db=FAISS.from_documents(documents[:30],OpenAIEmbeddings())

In [ ]:
query= 'An attention function can be described as mapping a query'
result = db.similarity_search(query)
result[0].page_content

In [ ]:
from langchain_community.llms import Ollama
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model='gpt-3.5-turbo')
llm

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
prompt=ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context. 
Think step by step before providing a detailed answer. 
I will tip you $1000 if the user finds the answer helpful. 
<context>
{context}
</context>
Question: {input}""")

In [ ]:
##Chain intro
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain=create_stuff_documents_chain(llm,prompt)

In [ ]:
retriever = db.as_retriever()
retriever

In [ ]:
##Retriever Chain
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)

In [ ]:
response = retrieval_chain.invoke({'input':"Scaled Dot-Product Attention"})

In [ ]:
response['answer']